In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

In [19]:
# load_dataset  焊死
model_path = "../SFT_model"
file_path = "../GRPO_data/training_prompt2.csv"  
df = pd.read_csv(file_path).iloc[200:]

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [21]:
system_prompt = """
You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
Follow this response template: <think>\nYour Thought Process (Why were these choices made?)\n</think>\n<answer>\n###Tables: (Related Table);\n###Columns: (Related Columns from Tables);\n</answer> 
"""
print(system_prompt)


You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
Follow this response template: <think>
Your Thought Process (Why were these choices made?)
</think>
<answer>
###Tables: (Related Table);
###Columns: (Related Columns from Tables);
</answer> 



In [22]:
def make_prompt(question, database_schema):
    conversation = []
    conversation0 = {}
    conversation1 = {}
    conversation0["role"] = "system"
    conversation0["content"] = system_prompt
    conversation1["role"] = "user"
    conversation1["content"] = f"<question>{question}</question>\n<database>{database_schema}</database>" + "\n"
    conversation.append(conversation0)
    conversation.append(conversation1)
    all_info = system_prompt + f"<question>{question}</question>\n<database>{database_schema}</database>" + "\n"
#     print(conversation2["content"])
    return conversation, all_info

In [23]:
trans_data = {}
prompts = []
solutions = []
max_token = 0
for index, row in df.iterrows():
    prompt, all_info = make_prompt(row["question"],row["database_schema"])
    prompts.append(prompt)
    solutions.append(row["target_schema"])
    token_len = len(tokenizer(all_info, add_special_tokens=False)["input_ids"])
    if  token_len >  max_token:
        max_token = token_len
trans_data["prompt"] = prompts
trans_data["solution"] = solutions

In [24]:
print(max_token)

2143


In [25]:
trans_data = pd.DataFrame(trans_data)
trans_data = Dataset.from_pandas(trans_data,split="train")
for sample in trans_data:
    print(sample)
    break  # 仅打印第一个样本

{'prompt': [{'content': "\nYou are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. \nFollow this response template: <think>\nYour Thought Process (Why were these choices made?)\n</think>\n<answer>\n###Tables: (Related Table);\n###Columns: (Related Columns from Tables);\n</answer> \n", 'role': 'system'}, {'content': '<question>What are the names of people in ascending order of weight?</question>\n<database>CREATE TABLE `entrepreneur` (\n  Entrepreneur_ID INT PRIMARY KEY,\n  People_ID INT REFERENCES people(People_ID),\n  Company TEXT,\n  Money_Requested REAL,\n  Investor TEXT\n);\n\nCREATE TABLE `people` (\n  People_ID INT PRIMARY KEY,\n  Name TEXT,\n  Height REAL,\n  Weight REAL,\n  Date_of_Birth TEXT\n);\n\n</database>\n', 'role': 'user'}], 'solution': '###Tables: people;\n###Columns: people.name, people.weight;\n'}
